In [1]:
cd ..

/home/nzilberstein/repository/constrained_sampling


/home/nzilberstein/anaconda3/envs/stable-dif/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# make sure you're logged in with \`huggingface-cli login\`
# import torch
# import argparse
# import os
from torchvision.utils import make_grid, save_image
import numpy as np
# from PIL import Image
import matplotlib.pyplot as plt
# import os
import hydra
from hydra import initialize, compose
from omegaconf import DictConfig, OmegaConf
import torchvision.transforms as transforms 
from torchvision.datasets import VisionDataset
from torch.utils.data import DataLoader
from typing import Callable, Optional
# import glob
import torch.nn as nn

from datasets import build_loader


import lpips
import torch

%load_ext autoreload
%autoreload 2

/home/nzilberstein/anaconda3/envs/stable-dif/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load config file
# with initialize(version_base=None, config_path="../_configs"):
#     cfg = compose(config_name='ddrmpp.yaml')

# # Define upsample operation
upsample = nn.Upsample(scale_factor=2, mode='nearest') 
# downsample = nn.AvgPool2d(2, 2)
# downsample = F.interpolate(scale_factor=0.5, mode='nearest')

# # Build loader
# loader = build_loader(cfg)
lpips_ = lpips.LPIPS(net='alex').cuda()

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/nzilberstein/anaconda3/envs/stable-dif/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nzilberstein/anaconda3/envs/stable-dif/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/nzilberstein/anaconda3/envs/stable-dif/lib/python3.12/site-packages/lpips/weights/v0.1/alex.pth


In [16]:
# algo = 'reddiff'
# algo = 'rsd_stable'
algo = 'rsd_stable_repulsion_gamma80'
max_img_id = '00018'

# img_path = f'/home/nzilberstein/repository/constrained_sampling/_exp/output/{algo}'
img_path = f'/home/nzilberstein/repository/constrained_sampling/_exp/output/{algo}/inp_large_box'
img_path_input = '/home/nzilberstein/repository/constrained_sampling/_exp/input/FFHQ'
coeff = 0.0
import os
import torch
from PIL import Image

dino = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
dino = dino.eval()
# Calculate diversity
cnt = 0
mean_psnr = 0
mean_lpips = 0
for item in os.listdir(img_path_input):
    if item.split('.')[0] < max_img_id:
        # Check if the item is a directory (folder)
        img_psld_torch_list = []    
        for img_id in os.listdir(os.path.join(img_path, item.split('.')[0])):
            # if img_id.split('_')[2] == "grid.png":
            #     continue
            img_id_path = os.path.join(img_path, item.split('.')[0], img_id)
            img = Image.open(img_id_path).convert('RGB')

            img_psld_torch = transforms.ToTensor()(img)
            img_psld_torch_list.append(img_psld_torch)
                # Transform to tensor the list
        img_psld_torch_list = torch.stack(img_psld_torch_list)
        # print(img_psld_torch_list.shape)
        # Compute similarity
        total_pair_wise_sim = 0.
        xo_list = dino(img_psld_torch_list)
        # del dino
        xo_list /= xo_list.norm(dim=-1, keepdim=True)
        # calculate the cosine similarity
        sim = (xo_list @ xo_list.T)
        # set the diagonal to be 0
        sim = sim - torch.diag(sim.diag())
        total_pair_wise_sim += sim.sum() / (sim.shape[0] * (sim.shape[0] - 1))
        torch.clear_autocast_cache()
        # print(total_pair_wise_sim)
                            
print(total_pair_wise_sim)

Using cache found in /home/nzilberstein/.cache/torch/hub/facebookresearch_dino_main


RuntimeError: stack expects each tensor to be equal size, but got [3, 512, 512] at entry 0 and [3, 516, 2058] at entry 7

In [7]:
1 - 0.9852

0.014800000000000035

## REDDIF

In [21]:
# algo = 'dps'
# algo = 'rsd_stable'
gamma = 80
algo = f'rsd_stable_repulsion_gamma{gamma}'
max_img_id = '00062'

# img_path = f'/home/nzilberstein/repository/constrai/ned_sampling/_exp/output/{algo}'
img_path = f'/home/nzilberstein/repository/constrained_sampling/_exp/output/{algo}/inp_large_box'
img_path_input = '/home/nzilberstein/repository/constrained_sampling/_exp/input/FFHQ'
coeff = 0.0
import os
import lpips
import os
import torch
from PIL import Image


if algo == 'rsd_stable' or algo == f'rsd_stable_repulsion_gamma{gamma}':
    # Calculate FID score
    cnt = 0
    mean_psnr = 0
    mean_lpips = 0
    for item in os.listdir(img_path_input):
        if item.split('.')[0] < max_img_id:
            print(item)
            # Check if the item is a directory (folder)
            img_true_path = os.path.join(img_path_input, item)
            # print(item)
            img_true = Image.open(img_true_path).convert('RGB')
            img_true_torch = transforms.ToTensor()(img_true)
            img_true_torch = upsample(img_true_torch.unsqueeze(0)).squeeze()
            for img_id in os.listdir(os.path.join(img_path, item.split('.')[0])):
                # if img_id.split('_')[2] == "grid.png":
                #     continue
                img_id_path = os.path.join(img_path, item.split('.')[0], img_id)
                img = Image.open(img_id_path).convert('RGB')

                img_psld_torch = transforms.ToTensor()(img)
                # print(img_psld_torch.shape)
                # img_psld_torch= nn.functional.interpolate(img_psld_torch.unsqueeze(0), size=(256, 256)).squeeze()

                mse = torch.mean((img_psld_torch - img_true_torch) ** 2, dim=(0, 1, 2))
                psnr = 10 * torch.log10(1 / (mse + 1e-10))
                # print(psnr)
                mean_psnr = mean_psnr + psnr

                lpips_val = lpips_(img_psld_torch.cuda(), img_true_torch.cuda())
                # print(lpips_val)
                mean_lpips = mean_lpips + lpips_val

                cnt = cnt + 1
else:
    # Calculate FID score
    cnt = 0
    mean_psnr = 0
    mean_lpips = 0
    for item in os.listdir(img_path_input):
        if item.split('.')[0] < max_img_id:
            # print(item)
            # Check if the item is a directory (folder)
            img_true_path = os.path.join(img_path_input, item)
            # print(item)
            img_true = Image.open(img_true_path).convert('RGB')
            img_true_torch = transforms.ToTensor()(img_true)
            # img_true_torch = upsample(img_true_torch.unsqueeze(0)).squeeze()
            for img_id in os.listdir(os.path.join(img_path, item.split('.')[0])):
                if len(img_id.split('.')[0].split('_')) == 2:
                    print(img_id)
                    img_id_path = os.path.join(img_path, item.split('.')[0], img_id)
                    img = Image.open(img_id_path).convert('RGB')

                    img_psld_torch = transforms.ToTensor()(img)

                    mse = torch.mean((img_psld_torch - img_true_torch) ** 2, dim=(0, 1, 2))
                    psnr = 10 * torch.log10(1 / (mse + 1e-10))
                    # print(psnr)
                    mean_psnr = mean_psnr + psnr

                    lpips_val = lpips_(img_psld_torch.cuda(), img_true_torch.cuda())
                    # print(lpips_val)
                    mean_lpips = mean_lpips + lpips_val

                cnt = cnt + 1


print(mean_psnr/cnt)
print(mean_lpips/cnt)

00024.png
00009.png
00043.png
00033.png
00001.png
00005.png
00038.png
00006.png
00056.png
00049.png
00018.png
00058.png
00010.png
00041.png
00045.png
00053.png
00055.png
00019.png
00015.png
00060.png
00023.png
00031.png
00017.png
00037.png
00003.png
00025.png
00044.png
00034.png
00048.png
00011.png
00052.png
00012.png
00040.png
00016.png
00007.png
00046.png
00026.png
00042.png
00054.png
00021.png
00028.png
00061.png
00032.png
00059.png
00039.png
00004.png
00022.png
00051.png
00013.png
00020.png
00057.png
00050.png
00035.png
00002.png
00047.png
00000.png
00014.png
00008.png
00027.png
00029.png
00036.png
00030.png
tensor(24.3240)
tensor([[[[0.1154]]]], device='cuda:0', grad_fn=<DivBackward0>)


In [ ]:
img_path = '/home/nzilberstein/Inverse/_exp/output_bip_small_nonaugm/RED_diff_nonaug'
it = 0
for item in os.listdir(img_path_input):
    # Check if the item is a directory (folder)
    if item.isnumeric():
        print(item)
        list_imgs = os.listdir(os.path.join(img_path_input, item))
        idx_img = np.random.choice(len(os.listdir(os.path.join(img_path_input, item))))
        img_true = Image.open(os.path.join(args.dir_path, item, list_imgs[idx_img])).convert('RGB')

        img_torch = transforms.ToTensor()(img_true)

        # Create folder if does not exist
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)

        save_image(img_torch, f'{dest_folder}/{it}.png')
    it = it + 1


In [ ]:
# /home/nicolas/red_diff_stable/PSLD/stable-diffusion/outputs/psld-samples-bip/samples/00002.png
from PIL import Image
import lpips
# img_path = '/home/nzilberstein/Inverse/_exp/output_rip/PSLD/samples/00003/00000.png'
img_id_single = '00010'
img_path = f'/home/nzilberstein/Inverse/PSLD/stable-diffusion/outputs/psld-samples-bip/samples/{img_id_single}/00000.png'
# img_path = f'/home/nzilberstein/Inverse/_exp/output_rip/PSLD/samples_500_iter/{img_id_single}/00000.png'
img = Image.open(img_path).convert('RGB')

img_path_reddif = f'/home/nzilberstein/Inverse/_exp/output_rip/RED_diff/10.0/x_hat_1.png'
# img_path_reddif = f'/home/nzilberstein/Inverse/_exp/output_rip/RED_diff/{img_id_single}/0.0/x_hat_1.png'
img_reddif = Image.open(img_path_reddif).convert('RGB')

img_true_path = f'/home/nzilberstein/Inverse/_exp/input/FFHQ/{img_id_single}.png'
# img_true_path = '/home/nzilberstein/PSLD/diffusion-posterior-sampling/data/samples/00003.png'
img_true = Image.open(img_true_path).convert('RGB')

img_deg = Image.open('/home/nzilberstein/Inverse/_exp/output_inv/x_deg.png').convert('RGB')
# Define upsample operation
upsample = nn.Upsample(scale_factor=2, mode='nearest') 

In [7]:
# algo = 'reddiff'

# img_path = f'/home/nzilberstein/repository/constrained_sampling/_exp/output/{algo}'
img_path = '/home/nzilberstein/threestudio/outputs/dreamfusion-if/A_stuffed_animal_that_is_frowning_is_on_a_skateboard@20240620-014941/gamma200_false_seed1_lr001_guidance100/save/it3512-test'
coeff = 0.0
import os
import torch
from PIL import Image

dino = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
dino = dino.eval()
# Calculate diversity
cnt = 0
mean_psnr = 0
mean_lpips = 0
img_psld_torch_list = [] 
for img_id in os.listdir(img_path):
        # Check if the item is a directory (folder)
       
    img_id_path = os.path.join(img_path, img_id)
    img = Image.open(img_id_path).convert('RGB')

    img_psld_torch = transforms.ToTensor()(img)
    
    img_psld_torch_list.append(img_psld_torch[:, :, :512])
                # Transform to tensor the list
img_psld_torch_list = torch.stack(img_psld_torch_list, dim = 0)
# print(img_psld_torch_list.shape)
# Compute similarity
total_pair_wise_sim = 0.
xo_list = dino(img_psld_torch_list)
# del dino
xo_list /= xo_list.norm(dim=-1, keepdim=True)
# calculate the cosine similarity
sim = (xo_list @ xo_list.T)
# set the diagonal to be 0
sim = sim - torch.diag(sim.diag())
total_pair_wise_sim += sim.sum() / (sim.shape[0] * (sim.shape[0] - 1))
torch.clear_autocast_cache()
        # print(total_pair_wise_sim)
                            
print(total_pair_wise_sim)

Using cache found in /home/nzilberstein/.cache/torch/hub/facebookresearch_dino_main


tensor(0.8567, grad_fn=<AddBackward0>)
